In [1]:
# imports
import requests
import json
import os
import platform
import time
import datetime
import pandas as pd
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

In [2]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

In [42]:
# blobcity index scraper
def url_scraper(start = 0,
                end = 0,
                windowSize = "1280,720",
                headless = True,
                quitOnEnd = True,
                waitBetweenPress = 30,
                waitUntilTimeout = 20,
                wait = 1,
               ):
    
    url = 'https://cloud.blobcity.com/#/ps/explore'
    store = '../data/repositories/blobcity/index/'
    
    pages = {}
    
    if start == 0 and end == 0:
        return None
    
    if start > end:
        return None
    
    
    # selenium version
    CHROMEDRIVER_PATH = './.selenium/chromedriver'
    WINDOW_SIZE = windowSize

    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
    driver.implicitly_wait(waitBetweenPress)
    driver.get(url)

    # wait for a given class to be loaded (javascript generated code)
    try:
        # not-found
        element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.CLASS_NAME, 'show-for-sr')))
        print ("Page is ready!")
    except TimeoutException:
        print ("class - loading took too much time!")
        

            
    running = True
    i = start
        
    # store first page too
    if start == 1:
        time.sleep(wait)
        html = driver.page_source
        print(i, len(html))
        store_data(html, store+str(i)+'.html')
        i += 1

    
    while running:
        
        pressLink = str(i)

        # press link
        if pressLink != '':
            try:
                print('link', pressLink)
                element = WebDriverWait(driver, waitUntilTimeout).until(EC.visibility_of_element_located((By.LINK_TEXT, pressLink)))
                element.click()
            except TimeoutException:
                print ("Loading took too much time!")

        html = driver.page_source
        print(i, len(html))
        time.sleep(wait)
        store_data(html, store+str(i)+'.html')
        
        i += 1
        if i > end:
            running = False

            
    if quitOnEnd:
        time.sleep(5)
        driver.close()
        print('done')



url_scraper(start = 1, end = 125, headless = False, quitOnEnd = True)

class - loading took too much time!
1 6785276
link 2
2 6786576
link 3
3 6786165
link 4
4 6785513
link 5
5 6786594
link 6
6 6785817
link 7
7 6785671
link 8
8 6786331
link 9
9 6786153
link 10
10 6787142
link 11
11 6785729
link 12
12 6785113
link 13
13 6785840
link 14
14 6786086
link 15
15 6787137
link 16
16 6786157
link 17
17 6785431
link 18
18 6785972
link 19
19 6786219
link 20
20 6785567
link 21
21 6785900
link 22
22 6785189
link 23
23 6785920
link 24
24 6786296
link 25
25 6786086
link 26
26 6785828
link 27
27 6786126
link 28
28 6786279
link 29
29 6786194
link 30
30 6786291
link 31
31 6786533
link 32
32 6784737
link 33
33 6786533
link 34
34 6785992
link 35
35 6785575
link 36
36 6786198
link 37
37 6785825
link 38
38 6785545
link 39
39 6785812
link 40
40 6786263
link 41
41 6785977
link 42
42 6786117
link 43
43 6786101
link 44
44 6785956
link 45
45 6786140
link 46
46 6786721
link 47
47 6786097
link 48
48 6786127
link 49
49 6785527
link 50
50 6786550
link 51
51 6786368
link 52
52 6786618
l

In [89]:
# scrape index

store = '../data/repositories/blobcity/index/'
fp_csv = '../data/database/blobcity_01_index.csv'


files = os.listdir(store)

def scrape_index(html):
    soup = BeautifulSoup(html, 'html.parser')
    result = []
    
    base_url = 'https://cloud.blobcity.com/'
    
    items = soup.select('nb-card')
    
    for item in items:
        meta = {}
        
        header = item.find('nb-card-header')
        meta['title'] = header.text.strip()
        meta['blobcity_link'] = base_url + header.find('a').get('href')
        
        body = item.find('nb-card-body')
        meta['description'] = body.find('div', class_='col-md-7').text.strip()
        meta['github_link'] = str(body.find('a'))
        meta['github_link'] = re.findall(r'<a[^>]* href="([^"]*)"', meta['github_link'])
        if len(meta['github_link']) > 0:
            meta['github_link'] = meta['github_link'][0]
        else:
            meta['github_link'] = ''
        
        footer = item.find('nb-card-footer')
        footer = footer.select('div.col-md-6')
        meta['tags'] = footer[0].text.strip()
        meta['tags'] = meta['tags'].split(' • ')
        dates = footer[1].text.strip()
        dates = dates.split(' \xa0\xa0  ')
        # 'dates': ['Created at: 11/25/20, 11:29 AM', 'Modified at : 11/25/20, 11:29 AM']
        if (len(dates) > 0):
            meta['created_at'] = dates[0].replace('Created at: ','').strip()
            meta['modified_at'] = dates[1].replace('Modified at : ','').strip()
            
            try:
                meta['created_at'] = str(datetime.datetime.strptime(meta['created_at'], '%m/%d/%y, %I:%M %p'))
                meta['modified_at'] = str(datetime.datetime.strptime(meta['modified_at'], '%m/%d/%y, %I:%M %p'))
            except:
                print('cannot convert to datetime', meta['created_at'], meta['modified_at'])
        else:
            meta['created_at'] = footer[1].text.strip()
            meta['modified_at'] = ''
        
        result.append(meta)

    return result

items = []
for file in files:
    print(file)
    html = load_data(os.path.join(store,file))
    if 'file not found' in html:
        print(html)
        break
    meta = scrape_index(html)
    #print(meta)
    items.extend(meta)
    #break
    
print(len(items), items[0])
#store_data(links, folder_base+folder+out, toJson=True)

df = pd.DataFrame(items)
print(df.shape)

# drop duplicates
df = df.drop_duplicates(['github_link'])
print(df.shape)

# drop nan
df.dropna(subset = ['github_link'])

print(df.shape)
print(df.head())
        
# drop columns
#df.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)

1.html
10.html
100.html
101.html
102.html
103.html
104.html
105.html
106.html
107.html
108.html
109.html
11.html
110.html
111.html
112.html
113.html
114.html
115.html
116.html
117.html
118.html
119.html
12.html
120.html
121.html
122.html
123.html
124.html
125.html
13.html
14.html
15.html
16.html
17.html
18.html
19.html
2.html
20.html
21.html
22.html
23.html
24.html
25.html
26.html
27.html
28.html
29.html
3.html
30.html
31.html
32.html
33.html
34.html
35.html
36.html
37.html
38.html
cannot convert to datetime  Modified at :
39.html
4.html
40.html
41.html
42.html
43.html
44.html
45.html
46.html
47.html
48.html
49.html
5.html
50.html
51.html
52.html
53.html
54.html
55.html
56.html
57.html
58.html
59.html
6.html
60.html
61.html
62.html
63.html
64.html
65.html
cannot convert to datetime  Modified at :
66.html
67.html
68.html
69.html
7.html
70.html
71.html
72.html
73.html
74.html
75.html
76.html
77.html
78.html
79.html
8.html
80.html
81.html
82.html
83.html
84.html
85.html
86.html
87.html
88